In [ ]:
import pandas as pd
import selenium
import time
import selenium.webdriver as webdriver
# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException


In [109]:
#1 # Go to webpage https://www.amazon.in/
# For e.g. If user input is ‘guitar’. Then search for guitars..

driver = webdriver.Edge("msedgedriver.exe")
url='https://www.amazon.in/'
driver.get(url)
time.sleep(10)
search_text = driver.find_element_by_id("twotabsearchtextbox").send_keys("Guitar")
search_text = driver.find_element_by_id("nav-search-submit-button").click()

In [110]:
#2 
inital = 0
end_page = 3
urls = []
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]
for page in range(inital,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href') # Scraping the url from webelement
            urls.append(url)                # Appending the url to urls list
        
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception Occured ")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page


for url in urls[:10]:
    driver.get(url)                                                        # Loading the webpage by url
    time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Guitar = pd.DataFrame.from_dict(prod_dict)
Guitar

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5 stars,930 ratings,"₹6,999.00",7 Days Replacement,"Friday, July 30",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Kadence Store,Kadence Acoustica Series Semi Acoustic Ash Woo...,3.9 out of 5 stars,604 ratings,"₹7,199.00",7 Days Replacement,"Friday, July 30",In stock.,Product Type : Acoustic Guitar Fretboard mater...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Brand: medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.9 out of 5 stars,472 ratings,"₹2,399.00",7 Days Replacement,"Thursday, July 29",In stock.,Material: Wood || Colour: Blue || Acoustic...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the Jixing Store,"Jixing Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.8 out of 5 stars,"1,791 ratings","₹2,499.00",7 Days Replacement,"Friday, July 30",In stock.,Brand: Jixing .Type: Acoustic Guitars-Basic .S...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5 stars,"11,416 ratings","₹2,649.00",7 Days Replacement,"Tuesday, July 27",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,Visit the JUAREZ Store,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...",4.0 out of 5 stars,"3,898 ratings","₹2,649.00",7 Days Replacement,Aug 1 - 3,Only 1 left in stock.,"Black glossy finish, number of frets: 18, acou...",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
6,Visit the JUAREZ Store,JUAREZ JRZ38C Right Handed Acoustic Guitar wit...,4.0 out of 5 stars,"2,568 ratings","₹2,649.00",7 Days Replacement,"Saturday, July 31",In stock.,Number of Frets: 18 || Acoustic guitar with ...,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-N...
7,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5 stars,"11,416 ratings","₹2,649.00",7 Days Replacement,"Tuesday, July 27",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
8,Brand: Intern,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",3.9 out of 5 stars,"6,241 ratings","₹2,290.00",7 Days Replacement,"Saturday, July 31",In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/Intern-INT-38C-Acoustic-...
9,Visit the JUAREZ Store,JUAREZ JRZ38C Right Handed Acoustic Guitar (Su...,4.0 out of 5 stars,"3,920 ratings","₹2,649.00",7 Days Replacement,Aug 1 - 4,"In stock on July 30, 2021.",Number of Frets: 18 || Acoustic guitar with ...,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...


In [108]:
#3
driver = webdriver.Edge("msedgedriver.exe")
url='https://images.google.com/'
driver.get(url)
time.sleep(10)
search_bar = driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')    
# Finding the search bar using it's xpath
search_bar.send_keys("fruits")     
# Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//button[@class="Tg7LZd"]')    
# Finding the xpath of search button
search_button.click()        # Clicking the search button
time.sleep(5)
for _ in range(500):#Scroll
    driver.execute_script("window.scrollBy(0,10000)")
time.sleep(5)
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
img_urls

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSt0K1iLAfz0W2NeZk0OW_Orah2AlvzURw8Tw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTsRLK-La6p9LPjx5MkCD7KjZ2PFQY8XNX0YA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYRn-Emn3lR4WiglVrDx69oghAW7W8Al-26g&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQlCy3oYbyuVFpu_1gPqRsxYmFnuhD8TyY33w&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTls-whrQZSb3M438SoCy-p-WTKgy85phUKjA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQIL2OwcR3akS_ATJiJTqQLnW_5aGcUHoIKTw&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRHuZrTbdci-8ElGzgIh_W6sJeV9VOBeT3NiA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT3Fd1Y-_ZITh_utQW_mnjwhAyyHRwGqst_rA&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcROYH-uaHxJxZllD4i4p-a6Jx_gDHTAudnBxQ&usqp=CAU',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvQngZEi

In [113]:
#4 
driver = webdriver.Edge("msedgedriver.exe")
url="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url)
time.sleep(10)
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

#scraping the Brand_Name 
BName=driver.find_elements_by_xpath("//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)

#scraping the Storage_RAM_ROM 
ram=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
#scraping the P_F_Camera 
PC=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)

#scraping the Display_size_Resolution 
DS=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
#scraping the ProcessorAndCores 
P=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
#scraping the Battery 
B=driver.find_elements_by_xpath("//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)

#scraping the Price 
price=driver.find_elements_by_xpath("//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
flipKart_phones=pd.DataFrame([])
flipKart_phones['Brand_Name']=Brand_Name[:15]
flipKart_phones['Storage_RAM_ROM']=Storage_RAM_ROM[:15]
flipKart_phones['Amount P_F_Camera']=P_F_Camera[:15]
flipKart_phones['Display_size_Resolution']=Display_size_Resolution[:15]
flipKart_phones['ProcessorAndCores']=ProcessorAndCores[:15]
flipKart_phones['Battery']=Battery[:15]
flipKart_phones['Price']=Price[:15]

flipKart_phones

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"realme C20 (Cool Grey, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,MediaTek Helio G35 Processor,5000 mAh Battery,"₹6,999"
1,"realme C20 (Cool Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 256 GB,8MP Rear Camera | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,MediaTek Helio G35 Processor,5000 mAh Battery,"₹6,999"
2,"REDMI 9i (Nature Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Lithium Polymer Battery,"₹8,299"
3,"OPPO Reno6 Pro 5G (Stellar Black, 256 GB)",12 GB RAM | 256 GB ROM,64MP + 8MP + 2MP + 2MP | 32MP Front Camera,16.64 cm (6.55 inch) Full HD+ Display,MediaTek Dimensity 1200 Processor,4500 mAh Lithium-ion Polymer Battery,"₹39,990"
4,"REDMI 9i (Midnight Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Lithium Polymer Battery,"₹8,299"
5,"realme Narzo 30 5G (Racing Silver, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,48MP + 2MP + 2MP | 16MP Front Camera,16.51 cm (6.5 inch) Full HD+ Display,MediaTek Dimensity 700 (MT6833) Processor,5000 mAh Battery,"₹15,999"
6,"REDMI 9 Prime (Matte Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP + 8MP + 5MP + 2MP | 8MP Front Camera,16.59 cm (6.53 inch) Full HD+ Display,MediaTek Helio G80 Processor,5020 mAh Battery,"₹9,999"
7,"OPPO Reno6 5G (Stellar Black, 128 GB)",8 GB RAM | 128 GB ROM,64MP + 8MP + 2MP | 32MP Front Camera,16.33 cm (6.43 inch) Full HD+ Display,MediaTek Dimensity 900 Processor,4300 mAh Lithium-ion Polymer Battery,"₹29,990"
8,"POCO M3 (Cool Blue, 64 GB)",6 GB RAM | 64 GB ROM | Expandable Upto 512 GB,48MP + 2MP + 2MP | 8MP Front Camera,16.59 cm (6.53 inch) Full HD+ Display,Qualcomm Snapdragon 662 Processor,6000 mAh Lithium-ion Polymer Battery,"₹11,499"
9,"REDMI 9i (Sea Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Lithium Polymer Battery,"₹8,299"


In [118]:
#5 
import re
driver = webdriver.Edge("msedgedriver.exe")
url="https://www.google.co.in/maps"
driver.get(url)
time.sleep(10)

search = driver.find_element_by_id("searchboxinput")                     
search.clear()                                           
time.sleep(2)
search.send_keys('London')                                                   
button = driver.find_element_by_id("searchbox-searchbutton")          
button.click()                                                          
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, \nLongitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

URL Extracted:  https://www.google.co.in/maps/place/London,+UK/@51.5287352,-0.3817882,10z/data=!3m1!4b1!4m5!3m4!1s0x47d8a00baf21de75:0x52963a5addd52a99!8m2!3d51.5073509!4d-0.1277583
Latitude = 51.5287352, 
Longitude = -0.3817882


In [117]:
      print("Latitude = {}, \nLongitude = {}".format(lat, lng))

Latitude = 51.5287352, 
Longitude = -0.3817882


In [115]:
url1="https://trak.in/india-startup-funding-investment-2015/"
driver = webdriver.Edge("msedgedriver.exe")
url="https://www.google.co.in/maps"
driver.get(url)
time.sleep(10)
Dates=[]
Company=[]
Industry=[]
Investor_Name=[]
Investment_Type=[]
Amount=[]

#scraping the company_name 
companies=driver.find_elements_by_xpath("//td[@class='column-3']")
for i in companies:
    if i.text is None :
        Company.append("--") 
    else:
        Company.append(i.text)

#scraping the Industry 
Ind=driver.find_elements_by_xpath("//td[@class='column-4']")
for i in Ind:
    if i.text is None :
        Industry.append("--") 
    else:
        Industry.append(i.text)

#scraping the Dates 
dt=driver.find_elements_by_xpath("//td[@class='column-2']")
for i in dt:
    if i.text is None :
        Dates.append("--") 
    else:
        Dates.append(i.text)

#scraping the Investor_Name 
IN=driver.find_elements_by_xpath("//td[@class='column-7']")
for i in IN:
    if i.text is None :
        Investor_Name.append("--") 
    else:
        Investor_Name.append(i.text)

#scraping the Investment_Type 
IT=driver.find_elements_by_xpath("//td[@class='column-8']")
for i in IT:
    if i.text is None :
        Investment_Type.append("--") 
    else:
        Investment_Type.append(i.text)

#scraping the Amount 
Price=driver.find_elements_by_xpath("//td[@class='column-9']")
for i in Price:
    if i.text is None :
        Amount.append("--") 
    else:
        Amount.append(i.text)
Funding_2nd_qtr=pd.DataFrame([])
Funding_2nd_qtr['Company']=Company
Funding_2nd_qtr['Industry']=Industry
Funding_2nd_qtr['Investor_Name']=Investor_Name
Funding_2nd_qtr['Amount Invested']=Amount
Funding_2nd_qtr['Specification']=Investment_Type
Funding_2nd_qtr['Dates']=Dates
Funding_2nd_qtr

KeyboardInterrupt: 